# Script to generate feature score importance

## Detailed in "Integrated data mining, genome-scale modeling, and machine learning for predicting yeast bioproduction"

### Description: Script takes a ML trained model (see ML_Pipeline files) and a dataset (test, validate or training set) and evaluates the accuracy. 

#### Procedure:
1. Shuffle the values in each feature. Exampe ATP Cost unshuffled values [1,2,6,8,...,10] --> shuffled [2,8,2,10,...,6]
2. Evalute the prediction accuracy with the shuffled features.
3. Repeat 1000 times.
4. Average each accuracy prediction to obtain the importance score.

#### Inputs:
1. Trained ML model object
2. Dataset

## Code

### Import necessary libraries

In [1]:
import pandas as pd
import pickle
import numpy as np

from sklearn.linear_model import ElasticNet,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as xgb
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler,StandardScaler,FunctionTransformer
from decimal import Decimal, ROUND_DOWN
from sklearn.metrics import accuracy_score

from collections import defaultdict
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import pearsonr, linregress
from sklearn.model_selection import train_test_split, validation_curve,GridSearchCV,learning_curve,cross_val_score,KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as MSLE
from sklearn.metrics import mean_poisson_deviance as MPD
from sklearn.metrics import mean_gamma_deviance as MGD

### Scoring functions

In [2]:
def rsquared(y_true, y_pred):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = linregress(y_true, y_pred)
    return r_value**2


## Import the data file inputs

In [3]:
## Trained ML object
with open('M21iYL.pickle','rb') as f:
    masterGrid = pickle.load(f)
    
## Encoded Dataset 
with open ('TESTData_part3.pickle','rb') as f:
    encodedData = pickle.load(f)    

scaledData = encodedData[0]
scaledData['thermo(kJ/g)'] = scaledData['inputThermo(kJ/L)']/scaledData['csConcTotal']

masterGrid = masterGrid[0]   

In [4]:
## Features used from data set in final model
# Used features in the final model.
cols_train__set = [
# 'dir_evo'
# ,'number_genes_deleted'
# ,'reactor_type'
'mw_Lipids'
,'pH'
# ,'product_deltaG\'o'
,'product_deltaGo'
,'foldCarbonFed'
,'product_name'
# ,'number_total_genes_overexp'
# ,'promoterEncoded(Sum)'
,'rxt_volume'
# ,'N2SourceEncoded(mean)'
# ,'N2SourceEncoded(max)'
# ,'N2_content'
# ,'N2_contentEncoded'
,'inputThermo(kJ/L)'
# ,'number_genes_mod'
# ,'strain_background'
# ,'oxygen'
,'FermentationTime'
,'atp_cost'
,'precursorsRequiredEncoded'
# ,'ccmPrecursorToProduct(FA)'###########
,'nadh_nadph_cost'
,'Pathway_enzymatic_steps'
#,'no_c'
# ,'ccm_precursor_deltaGo'
,'averageThermBarrier'
# ,'totalThermBarrier'
# ,'csConcTotal'
,'media'
,'number_genes_het'
,'number_native_genes_overexp'
# ,'integrationSiteEncoded(Sum)'

,'Product_titer(g/L)'
,'Product_rate(g/L/h)'
,'Product_yield(g/gC)'#,

 ,'ATP_iYLI647'
 ,'NADPH_iYLI647'
 # ,'EMP_iYLI647'
 ,'PPP_iYLI647'
 ,'TCA_iYLI647'
 , 'PrdtYield_iYLI647'
 # ,'O2Uptake_iYLI647'
 # ,'Biomass_iYLI647'
]


# targets for ML, drop from feature list before training.
target_cols_todrop = [
'Product_yield(g/gC)'
,'Product_rate(g/L/h)'
,'Product_titer(g/L)'
]


useful_cols = []
useful_cols.extend(cols_train__set)
data = scaledData.loc[:,useful_cols]

for column in data:
    data[column] = data[column].astype(np.float32)


In [5]:
#for evaluating models trained with dummy variables for the product class

# 0 - no 
# 1 - yes
dummyOption = 0

#drop the dummy variables from the output metric list
dummy_drop = ['product_name','Product_titer(g/L)','Product_rate(g/L/h)','Product_yield(g/gC)']
dummy_cols_train__set = list(set(cols_train__set)-set(dummy_drop))

In [6]:
#uncomment following line to test validate, training split. For test data, comment line.
# train, test = train_test_split(data, test_size = 0.20, random_state = 566,stratify=data['product_name'])

test = data #comment out if want to use validate, training data, generated in line above
train = data #comment out if want to use validate, training data, generated in line above
x_train = train.copy()
x_test = test.copy()
y_test = test.copy()

if dummyOption == 1:
    toCont = ['product_name']
    x_train = pd.get_dummies(train,columns=toCont)
    x_test = pd.get_dummies(test,columns=toCont)
    y_test = pd.get_dummies(test,columns=toCont)

# Drop the output metrics from training set.
for target1 in target_cols_todrop:
    x_train.drop(target1,axis=1,inplace=True)
    x_test.drop(target1,axis=1,inplace=True)


x_testData = x_test.copy()
target = 'Product_titer(g/L)'


#### Initialize the storage for the output metrics

In [7]:
r2_ = defaultdict(list)
rsquareds = defaultdict(list)
r2_improvement = defaultdict(list)
r2_improvement_percent = defaultdict(list)
mse_ = defaultdict(list)
mae_ = defaultdict(list)
mae_scores = defaultdict(list)
mae_improvement = defaultdict(list)
mae_improvement_percent = defaultdict(list)
number_list = []

In [8]:
#generate the initial reuslts

y_prediction = np.exp(masterGrid[target].predict(x_testData))
y_actual = y_test[target]
r2_acc = rsquared(y_prediction,y_actual)
mae_acc = mae(y_prediction,y_actual)
mse_acc = mse(y_prediction,y_actual)

#were dummy variables used?
if dummyOption==0:
    
    #list of features to sort through
    names=list(x_testData)
    
    #shuffle through features 1000 times
    for iterations in range(0,1000):
        #shuffle through each feature
        for w in range(x_testData.shape[1]):
            
            #copy of the initial data
            x_t = x_testData.copy() 
            
            #random permutation of values
            x_t[names[w]]= np.random.permutation(x_t[names[w]].values)

            #prediction and metrics
            prediction = np.exp(masterGrid[target].predict(x_t))
            r2_shuff_acc = rsquared(y_actual, prediction)
            r2_[names[w]] = r2_shuff_acc
            rsquareds[names[w]].append(abs((r2_acc-r2_shuff_acc)/r2_acc))
            mae_shuff_acc = mae(y_actual,prediction)
            mae_[names[w]] = mae_shuff_acc
            mae_scores[names[w]].append(abs((mae_acc-mae_shuff_acc)/mae_acc))
            mse_shuff_acc = mse(y_actual,prediction)
            mse_[names[w]] = mse_shuff_acc

        if (iterations%25)==0:
            print (iterations)
else:
    
    #shuffle through features 1000 times    
    for iterations in range(0,1000):
        
        #shuffle through each feature
        for w in dummy_cols_train__set:

            #copy of the initial data
            x_t = x_testData.copy()

            #random permutation of values
            x_t[[w]]= np.random.permutation(x_t[[w]].values)

            #prediction and metrics
            prediction = np.exp(masterGrid[target].predict(x_t))
            r2_shuff_acc = rsquared(y_actual, prediction)
            r2_[str(w)] = r2_shuff_acc
            rsquareds[str(w)].append(abs((r2_acc-r2_shuff_acc)/r2_acc))
            mae_shuff_acc = mae(y_actual,prediction)
            mae_[str(w)] = mae_shuff_acc
            mae_scores[str(w)].append(abs((mae_acc-mae_shuff_acc)/mae_acc))
            mse_shuff_acc = mse(y_actual,prediction)
            mse_[str(w)] = mse_shuff_acc
        
        if (iterations%25)==0:
            print (iterations)


0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975


In [9]:
print('Normal Metrics')
print('R2',r2_acc,'MAE',mae_acc,'MSE',mse_acc)

Normal Metrics
R2 0.8882798308339088 MAE 2.4386030483458137 MSE 44.11033246713216


In [10]:
test = (sorted([(round(np.mean(score), 4), feat) for feat, score in mae_.items()], reverse=False))
print('features scores, sorted:\n')
print(test)

features scores, sorted:

[(3.3412, 'number_native_genes_overexp'), (3.4653, 'mw_Lipids'), (3.4719, 'TCA_iYLI647'), (3.5408, 'FermentationTime'), (3.5851, 'number_genes_het'), (3.7023, 'precursorsRequiredEncoded'), (3.7126, 'pH'), (3.7812, 'nadh_nadph_cost'), (3.8677, 'rxt_volume'), (3.9937, 'foldCarbonFed'), (4.0428, 'media'), (4.0682, 'product_deltaGo'), (4.1259, 'atp_cost'), (4.2234, 'averageThermBarrier'), (4.2415, 'PrdtYield_iYLI647'), (4.3509, 'product_name'), (4.3669, 'NADPH_iYLI647'), (4.4422, 'Pathway_enzymatic_steps'), (4.5654, 'ATP_iYLI647'), (5.2153, 'inputThermo(kJ/L)'), (5.2479, 'PPP_iYLI647')]
